In [32]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse


def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_all_images(url):
    """
    Returns all image URLs on a single `url`
    """
    soup = bs(requests.get(url).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)
        # remove URLs like '/hsts-pixel.gif?c=3.2.5'
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # finally, if the url is valid
        if is_valid(img_url):
            urls.append(img_url)
    return urls

In [35]:
def download(url, pathname):
    """
    Downloads a file given an URL and puts it in the folder `pathname`
    """
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    buffer_size = 1024
    response = requests.get(url, stream=True)

    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))

    # get the file name
    filename = os.path.join(pathname, pathname+'__'+url.split("/")[-2])

    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(buffer_size), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))


def main(url, path):
    # get all images
    imgs = get_all_images(url)
    for img in imgs:
        # for each img, download it
        download(img, path) 
        
def supermain(url):
    # get dates for all sub-urls for full image download
    dates = get_all_images(url)
    for date in dates:
        # for each date of event
        fname = date.split("/")[-2]
        # remove extension
        pname = fname.rsplit('.', 1)[0]
        # new url from date
        newurl = 'http://meglab.wikidot.com/'+pname+'photos'
        # download all pics from new url
        main(newurl, pname)
    

In [36]:
supermain('http://meglab.wikidot.com/2018-2019')


Extracting images: 100%|██████████| 36/36 [00:00<00:00, 32882.17it/s]

Extracting images: 100%|██████████| 2/2 [00:00<00:00, 7019.76it/s]



Extracting images: 100%|██████████| 7/7 [00:00<00:00, 16786.81it/s]
































Extracting images: 100%|██████████| 11/11 [00:00<00:00, 14087.74it/s]





























































































Extracting images: 100%|██████████| 6/6 [00:00<00:00, 5300.30it/s]




















Extracting images: 100%|██████████| 5/5 [00:00<00:00, 9074.65it/s]












Extracting images: 100%|██████████| 7/7 [00:00<00:00, 7937.31it/s]












Extracting images: 100%|██████████| 6/6 [00:00<00:00, 13155.16it/s]
























Extracting images: 100%|██████████| 2/2 [00:00<00:00, 5360.13it/s]



Extracting images: 100%|██████████| 3/3 [00:00<00:00, 8981.38it/s]





Extracting images: 100%|██████████| 5/5 [00:00<00:00, 11143.21it/s]












Extracting images: 100%|██████████| 